<a href="https://colab.research.google.com/github/Chintan3627/nlp-projects/blob/main/text_Summerization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Aug 15 05:16:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    35W /  70W |  14247MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
# !pip install datasets transformers[sentencepiece] sacrebleu -q

In [ ]:
from transformers import BartTokenizer, TFAutoModelForSeq2SeqLM

# Load the pre-trained model and tokenizer
model_name = "facebook/bart-large-cnn"
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large-cnn.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset


In [ ]:
  datasets = load_dataset("samsum")

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] ,padding='max_length', max_length = 1024,return_tensors="tf", truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'],padding='max_length', max_length = 128, truncation = True ,return_tensors="tf")

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [ ]:
dataset_samsum_tf = datasets.map(convert_examples_to_features, batched = True)


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
dataset_samsum_tf

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model,return_tensors='tf')

In [ ]:
seq2seq_data_collator

DataCollatorForSeq2Seq(tokenizer=BartTokenizer(name_or_path='facebook/bart-large-cnn', vocab_size=50265, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True), model=<transformers.models.bart.modeling_tf_bart.TFBartForConditionalGenerat

In [ ]:
batch_size = 2
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

In [ ]:
train_dataset = model.prepare_tf_dataset(
    dataset_samsum_tf["test"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=seq2seq_data_collator,
)

In [ ]:
dataset_samsum_tf["test"]


Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 819
})

In [ ]:
validation_dataset = model.prepare_tf_dataset(
    dataset_samsum_tf["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=seq2seq_data_collator,
)

In [ ]:
generation_dataset = model.prepare_tf_dataset(
    dataset_samsum_tf["test"],
    batch_size=8,
    shuffle=False,
    collate_fn=seq2seq_data_collator,
)

In [ ]:
from transformers import AdamWeightDecay
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=1)

409/409 [==============================] - 721s 2s/step - loss: 1.6041 - val_loss: 1.5774


In [ ]:
model.save_pretrained("summarization_model/")

In [ ]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.json',
 'tokenizer/merges.txt',
 'tokenizer/added_tokens.json')

In [ ]:
!zip -r summarization_model.zip summarization_model


  adding: summarization_model/ (stored 0%)
  adding: summarization_model/tf_model.h5 (deflated 7%)
  adding: summarization_model/config.json (deflated 62%)
  adding: summarization_model/generation_config.json (deflated 47%)


In [ ]:
  !zip -r summarization_model_tokenizer.zip tokenizer


  adding: tokenizer/ (stored 0%)
  adding: tokenizer/vocab.json (deflated 68%)
  adding: tokenizer/tokenizer_config.json (deflated 81%)
  adding: tokenizer/special_tokens_map.json (deflated 85%)
  adding: tokenizer/merges.txt (deflated 53%)


In [ ]:
from google.colab import files

# Download the zip file
# files.download('/content/summarization_model.zip')
